In [1]:
import os
import random
import cv2
from cvzone.FaceMeshModule import FaceMeshDetector
import cvzone

In [2]:
cap = cv2.VideoCapture(0)    # 비디오 캡처 객체 생성 (0은 기본 카메라)
cap.set(3, 1280)    # 가로길이
cap.set(4, 720)    # 세로 길어

True

In [3]:
detector = FaceMeshDetector(maxFaces = 1)    # 최대 1개의 얼굴 감지
idList = [0, 17, 78, 292]    # 사용할 FaceMesh 지점의 ID 리스트 생성

In [4]:
# 먹을 수 있는 오브젝트의 이미지를 로드
folderEatable = 'Objects/eatable'
listEatable = os.listdir(folderEatable)
eatables = []
for object in listEatable:
    eatables.append(cv2.imread(f'{folderEatable}/{object}', cv2.IMREAD_UNCHANGED))

In [5]:
# 먹을 수 없는 오브젝트의 이미지 로드
folderNonEatable = 'Objects/noneatable'
listNonEatable = os.listdir(folderNonEatable)
nonEatables = []
for object in listNonEatable:
    nonEatables.append(cv2.imread(f'{folderNonEatable}/{object}', cv2.IMREAD_UNCHANGED))

In [6]:
currentObject = eatables[0]    # 연재 오브젝트를 먹을 수 있는 첫번째 오브젝트로 설정
pos = [300, 0]    # 오브젝트 초기 위치
speed = 5    # 오브젝트 초기 속도
count = 0    # 게임 점수 0점 초기화
global isEatable    # 먹을 수 있는지 여부 나타내는 글로벌 변수
isEatable = True    # 오브젝트는 초기에 먹을 수 있음
gameOver = False    # 게임 오버 상태 False로 초기화

In [7]:
def resetObject():    # 오브젝트 리셋하는 함수
    global isEatable
    pos[0] = random.randint(100, 1180)    # 오브젝트의 x좌표 랜덤 상성
    pos[1] = 0 # 오브젝트의 y 좌표를 0으로 설정
    randNo = random.randint(0,2)    # 0, 1, 2중 랜덤한 숫자를 생성
    if randNo == 0:
        currentObject = nonEatables[random.randint(0, 3)]    # 먹을 수 없는 오브젝트 중 하나 랜덤하게 선택
        isEatable = True    # 오브젝트 먹을 수 있음
    
    return currentObject

In [8]:
while True:  # 무한 루프
    success, img = cap.read()  # 카메라로부터 이미지를 캡처

    if gameOver is False:  # 게임 오버 상태가 아니라면,
        img, faces = detector.findFaceMesh(img, draw=False)  # FaceMesh를 찾아서 그리지 않음

        img = cvzone.overlayPNG(img, currentObject, pos)  # 현재 오브젝트를 이미지에 오버레이
        pos[1] += speed  # 오브젝트의 y 좌표를 속도만큼 증가시킴

        if pos[1] > 520:  # 오브젝트가 화면 하단에 도달하면,
            currentObject = resetObject()  # 오브젝트를 리셋

        if faces:  # 얼굴이 감지되면,
            face = faces[0]  # 첫 번째 얼굴을 선택

            up = face[idList[0]]  # 얼굴의 위쪽 지점 선택
            down = face[idList[1]]  # 얼굴의 아래쪽 지점 선택

            for id in idList:  # idList의 모든 id에 대해,
                cv2.circle(img, face[id], 5, (255, 0, 255), 5)  # 얼굴 지점에 원을 그림
            cv2.line(img, up, down, (0, 255, 0), 3)  # 얼굴의 위쪽과 아래쪽 지점 사이에 선을 그림
            cv2.line(img, face[idList[2]], face[idList[3]], (0, 255, 0), 3)  # 얼굴의 두 가운데 지점 사이에 선을 그림

            upDown, _ = detector.findDistance(face[idList[0]], face[idList[1]])  # 위쪽과 아래쪽 지점 사이의 거리 계산
            leftRight, _ = detector.findDistance(face[idList[2]], face[idList[3]])  # 두 가운데 지점 사이의 거리 계산

            ## Distance of the Object
            cx, cy = (up[0] + down[0]) // 2, (up[1] + down[1]) // 2  # 위쪽과 아래쪽 지점의 중간 좌표 계산
            cv2.line(img, (cx, cy), (pos[0] + 50, pos[1] + 50), (0, 255, 0), 3)  # 중간 좌표와 오브젝트 사이에 선을 그림
            distMouthObject, _ = detector.findDistance((cx, cy), (pos[0] + 50, pos[1] + 50))  # 중간 좌표와 오브젝트 사이의 거리 계산
            print(distMouthObject)  # 계산된 거리를 출력

            # Lip opened or closed
            ratio = int((upDown / leftRight) * 100)  # 입이 열려있는지 여부를 결정하는 비율 계산
            # print(ratio)  # 계산된 비율을 출력
            if ratio > 60:  # 비율이 60보다 크면,
                mouthStatus = "Open"  # 입이 열려있음
            else:  # 비율이 60보다 작거나 같으면,
                mouthStatus = "Closed"  # 입이 닫혀있음
            cv2.putText(img, mouthStatus, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 0, 255), 2)  # 입의 상태를 이미지에 텍스트로 표시

            if distMouthObject < 100 and ratio > 60:  # 거리가 100 미만이고, 입이 열려있으면,
                if isEatable:  # 오브젝트가 먹을 수 있는 경우,
                    currentObject = resetObject()  # 오브젝트를 리셋하고,
                    count += 1  # 점수를 1점 늘림
                else:  # 오브젝트가 먹을 수 없는 경우,
                    gameOver = True  # 게임 오버
        cv2.putText(img, str(count), (1100, 50), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 0, 255), 5)  # 현재 점수를 이미지에 텍스트로 표시
    else:  # 게임 오버 상태라면,
        cv2.putText(img, "Game Over", (300, 400), cv2.FONT_HERSHEY_PLAIN, 7, (255, 0, 255), 10)  # "Game Over" 텍스트를 이미지에 표시

    cv2.imshow("Image", img)  # 이미지를 화면에 표시
    key = cv2.waitKey(1)  # 사용자의 키 입력을 기다림

    if key == ord('r'):  # 'r' 키를 누르면,
        resetObject()  # 오브젝트를 리셋하고,
        gameOver = False  # 게임 오버 상태를 해제하며,
        count = 0  # 점수를 0으로 리셋하고,
        currentObject = eatables[0]  # 첫 번째 먹을 수 있는 오브젝트를 현재 오브젝트로 설정
        pos = [300, 0]  # 오브젝트의 위치를 초기 위치로 설정

    if key == 27:  # ESC 키를 누르면,
        break  # 무한 루프를 빠져나옴
        
cap.release()  # 비디오 캡처 객체를 해제
cv2.destroyAllWindows()  # 모든 창을 닫음

544.063415421401
541.9012825229333
539.6665637224526
534.7990276730129
531.2071159161933
525.9515186782903
521.0959604525831
515.3736508592576
511.51930559852775
507.1853704514751
505.95849632158564
503.95436301316016
501.97410291767045
497.298702994488
493.6172606382398
489.54672912807825
484.59983491536605
480.60794833210986
473.8143096192853
469.97233961159884
464.0797345284536
461.5333140738597
457.1214280691729
453.7455233938953
449.5887009256349
445.7140338827127
441.58351418502934
438.2921400162225
435.588108193968
434.1946568072896
441.6480499221071
440.01022715386966
441.6480499221071
441.97963754001154
439.30058046854435
435.0287346831241
429.81507651547076
422.6121153019634
411.5167068297471
397.1662120573703
380.5693103759156
364.04532684818247
346.2080299473136
331.3547947442439
321.06852851065923
314.58226269133485
309.40103425812913
307.2621030976648
304.138126514911
294.11052344314373
273.2471408816568
243.82165613415063
221.0271476538572
209.4612135933524
202.230067002

UnboundLocalError: local variable 'currentObject' referenced before assignment